In [2]:
import random

# Чтение данных из файла
with open('data.txt', 'r') as file:
    data = [int(line.strip()) for line in file]


#m - number of machines, n - number of all tasks (should be n%m ==0), 
#a, b are start and end values for creating random time of tasks
# def creator(data, m = 10) -> dict:
#     machines = [[] for x in range(m)]
#     n = len(data)
#     data_sort = data.copy()
#     data_sort.sort()
#     tasks_list = {}
#     #print(data_sort)
#     for i in range(m): #количество машин
#         for j in range(i, n, n//m):
#             machines[i].append(data_sort[j]) #распределяет 1 элемент из первых (n//m), потом второй элемент n//m+i из (n//m,2*n//m) etc
#     #print(machines)
#     tasks_dict = dict(enumerate(machines))
#     return tasks_dict
# tasks_dict = creator(data)

In [12]:
import copy
def creator(data, m = 10) -> list:
    machines = [[] for x in range(m)]
    n = len(data)
    data_sort = data.copy()
    data_sort.sort()
    tasks_list = []
    for i in range(m): #количество машин
        for j in range(i, n, n//m):
            machines[i].append(data_sort[j]) #распределяет 1 элемент из первых (n//m), потом второй элемент n//m+i из (n//m,2*n//m) etc
    return machines
#print(creator(data))

def objective_function(tuple_) -> int:
    list1, list2 = tuple_
    return abs(sum(list1) - sum(list2))

def get_neighbors(tuple_):
    neighbors = []
    list_1, list_2 = tuple_
    for i in range(len(list_1)):
        for j in range(i + 1, len(list_2)):
            # глубокие копии списков (не нужны) перед каждым изменением
            list1, list2 = copy.deepcopy(list_1), copy.deepcopy(list_2)
            list1[i], list2[j] = list2[j], list1[i]
            neighbors.append((list1, list2))
        neighbors.append(replacing(list1, list2))
    return neighbors

def replacing(list1, list2):
    semi_diff = (sum(list1) - sum(list2))/2
    if semi_diff > 0:
        big, small = list1, list2 
    else:
        big, small = list2, list1
    # big.sort()
    # small.sort()
    smallest_dist = 10**10
    for i in range(len(big)):
        distance = abs(big[i] - semi_diff)
        if distance < smallest_dist:
            smallest_dist = distance
            near = big[i]
    small.append(near)
    big.remove(near)
    return (list1, list2)

def tabu_search (gl_list, max_iterations, tabu_list_size):
    best_solution_gl = gl_list#copy.deepcopy(gl_list)   
    #должна произвойти выборка 2 листов
    best_solution = None
    tabu_list = []
    
    for p in range(len(gl_list)):
        for x in range(len(gl_list)-p):        
            current_solution = [gl_list[x][:], gl_list[x+p][:]]
            #tabu_list = [] #должен ли он в цикле находиться? 
            
            for _ in range(max_iterations):
                neighbors = get_neighbors(current_solution) #лист кортежей по два соседа
                best_neighbor = None
                best_neighbor_fitness = float('inf')
        
                for neighbor in neighbors:
                    if neighbor not in tabu_list:
                        neighbor_fitness = objective_function(neighbor)
                        if neighbor_fitness < best_neighbor_fitness:
                            best_neighbor = neighbor
                            best_neighbor_fitness = neighbor_fitness
        
                if best_neighbor is None:
                    # No non-tabu neighbors found,
                    # terminate the search
                    break
    
                current_solution = best_neighbor
                tabu_list.append(best_neighbor)
                if len(tabu_list) > tabu_list_size:
                    # Remove the oldest entry from the
                    # tabu list if it exceeds the size
                    tabu_list.pop(0)
               # print(objective_function(best_neighbor, gl_list), objective_function((best_solution_gl[x][:], best_solution_gl[-x][:]), gl_list))
                if objective_function(best_neighbor) < objective_function((best_solution_gl[x][:], best_solution_gl[x+p][:])):
                    # Update the best solution if the
                    # current neighbor is better
                    best_solution = best_neighbor 
                    #мы получили лучшее решение для пары, 
                    # теперь нам надо будет взять пару и вставить в массив
                    # при этом как-то обработать повторяющиеся? 
                    # пока я делаю х и -х, повторов не будет, но решение не лечшее
                    # мы получаем для каждой ...
            if best_solution is not None:  # Проверяем, что best_solution был обновлен
                best_solution_gl[x], best_solution_gl[x+p] = best_solution
    return best_solution_gl
###########################3
# def another(gl_list, max_iterations, tabu_list_size):
#     best_solution_gl = copy.deepcopy(gl_list)   
#     #должна произвойти выборка 2 листов
#     best_solution = None
#     tabu_list = []
#     for p in range(9):
#         for x in range(len(gl_list)-1):        
#             current_solution = [gl_list[x][:], gl_list[(x+p)%10][:]]
#              #должен ли он в цикле находиться? 
            
#             for _ in range(max_iterations):
#                 neighbors = replacing(current_solution[0], current_solution[1])
#                 best_neighbor = None
#                 best_neighbor_fitness = float('inf')
               #  neighbor = neighbors
               #  # for neighbor in neighbors:
               #  if neighbor not in tabu_list:
               #      neighbor_fitness = objective_function(neighbor)
               #      if neighbor_fitness < best_neighbor_fitness:
               #          best_neighbor = neighbor
               #          best_neighbor_fitness = neighbor_fitness
        
               #  if best_neighbor is None:
               #      # No non-tabu neighbors found,
               #      # terminate the search
               #      break
    
               #  current_solution = best_neighbor
               #  tabu_list.append(best_neighbor)
               #  if len(tabu_list) > tabu_list_size:
               #      # Remove the oldest entry from the
               #      # tabu list if it exceeds the size
               #      tabu_list.pop(0)
               # # print(objective_function(best_neighbor, gl_list), objective_function((best_solution_gl[x][:], best_solution_gl[-x][:]), gl_list))
               #  if objective_function(best_neighbor) < objective_function((best_solution_gl[x][:], best_solution_gl[(x+p)%10][:])):
               #      # Update the best solution if the
               #      # current neighbor is better
               #      best_solution = best_neighbor 
                    #мы получили лучшее решение для пары, 
                    # теперь нам надо будет взять пару и вставить в массив
                    # при этом как-то обработать повторяющиеся? 
                    # пока я делаю х и -х, повторов не будет, но решение не лечшее
                    # мы получаем для каждой ...
            # if best_solution is not None:  # Проверяем, что best_solution был обновлен
            #     best_solution_gl[x], best_solution_gl[(x+p)%10] = best_solution

            
###################################         
    return best_solution_gl
qqq = tabu_search(creator(data), 1000, 10)
www = creator(data)
print(max(sum(x) for x in www), min(sum(x) for x in www), max(sum(x) for x in qqq), min(sum(x) for x in qqq))
for i in range(len(qqq)):
    # if www[i]!=qqq[i]:
        # print(sum(www[i]), www[i])
        print(sum(qqq[i]), qqq[i])
print('------------')
for i in range(len(www)):
    print(sum(www[i]), www[i])

537 457 501 490
497 [89, 59, 85, 96, 30, 24, 35, 41, 38]
497 [89, 59, 85, 96, 30, 24, 35, 41, 38]
496 [95, 91, 48, 41, 60, 21, 32, 19, 18, 18, 14, 15, 23, 1]
498 [85, 96, 89, 65, 35, 70, 58]
501 [62, 63, 97, 73, 56, 69, 68, 13]
490 [77, 58, 33, 73, 73, 28, 51, 25, 3, 12, 4, 13, 12, 28]
500 [38, 51, 51, 58, 58, 85, 35, 70, 32, 1, 21]
495 [38, 51, 51, 58, 58, 95, 32, 60, 30, 1, 21]
496 [38, 51, 51, 58, 60, 96, 30, 58, 32, 1, 21]
497 [51, 51, 58, 58, 86, 58, 65, 70]
------------
457 [1, 12, 18, 28, 41, 56, 59, 71, 80, 91]
465 [1, 12, 18, 30, 41, 56, 60, 73, 83, 91]
477 [2, 13, 19, 32, 44, 57, 61, 73, 85, 91]
483 [3, 13, 20, 32, 47, 57, 62, 73, 85, 91]
491 [4, 14, 21, 32, 47, 57, 63, 76, 85, 92]
502 [5, 14, 23, 33, 48, 57, 65, 77, 86, 94]
512 [6, 15, 24, 33, 50, 58, 68, 77, 86, 95]
519 [7, 15, 25, 35, 51, 58, 69, 77, 86, 96]
523 [7, 15, 26, 35, 51, 58, 70, 77, 87, 97]
537 [8, 17, 28, 38, 51, 58, 70, 79, 89, 99]
